#GET A ROOM : ML HACKATHON
Identify the habitability score of a property

#Problem Statement
Finding the correct property to live in is a crucial task while moving to a new city/location. An inappropriate property can make our life miserable. Can AI help us find better places?

#Task
You have given a relevant dataset about various properties in the USA. Your task is to identify the habitability score of the property.   

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
ohe = OneHotEncoder()
sc = StandardScaler()

In [3]:
import os
os.listdir("/kaggle/input/usa-property-dataset/dataset")

['sample_submission.csv', 'train.csv', 'test.csv']

In [4]:
train = pd.read_csv("/kaggle/input/usa-property-dataset/dataset/train.csv")

In [5]:
test = pd.read_csv("/kaggle/input/usa-property-dataset/dataset/test.csv")

In [ ]:
train.shape,test.shape

In [ ]:
print(train.isnull().sum())
print(test.isnull().sum())

In [ ]:
y = train.iloc[:,-1]
train.drop(train.columns[-1],axis=1,inplace=True)

In [ ]:
final = pd.concat([train,test])
final.reset_index(inplace=True)
final=final.iloc[:,1:]
final.shape

#Data Cleaning : Approach 1

In [ ]:
missed_col =["Number_of_Windows","Furnishing","Frequency_of_Powercuts","Crime_Rate","Dust_and_Noise"]

In [ ]:
miss = final.loc[:,missed_col]
miss

In [ ]:
final.corr().plot(kind="bar",figsize=(20,6))

In [ ]:
def lore(df,miss):
  from sklearn.linear_model import LogisticRegression
  lr = LogisticRegression()
  lr.fit(df.iloc[:,[1]],df.iloc[:,[0]])
  pred = lr.predict(miss)
  return pred

In [ ]:
plt.scatter(final["Number_of_Doors"],final["Number_of_Windows"])

In [ ]:
ddf = final.iloc[:,[3,4]]
tbs = ddf.loc[ddf["Number_of_Windows"].notnull()]
tp =  ddf.loc[ddf["Number_of_Windows"].isnull()]
ts = tp.iloc[:,1]
pred = lore(tbs,pd.DataFrame(ts))
tp.iloc[:,0]=pred
tp

In [ ]:
windows_filled = pd.concat([tbs,tp]).sort_index()

In [ ]:
plt.scatter(final.iloc[:,6],final.iloc[:,-1])

In [ ]:
ddf = final.iloc[:,[6,-1]]
tbs = ddf.loc[ddf["Frequency_of_Powercuts"].notnull()]
tp =  ddf.loc[ddf["Frequency_of_Powercuts"].isnull()]
ts = tp.iloc[:,1]
pred = lore(tbs,pd.DataFrame(ts))
tp.iloc[:,0]=pred
tp

In [ ]:
fop_filled = pd.concat([tbs,tp]).sort_index()
fop_filled

In [ ]:
for a in final.iloc[:,[5,10,11]]:
  print(final[a].unique())

In [ ]:
final2= final.copy()
a={"Unfurnished":0,"Semi_Furnished":1,"Fully Furnished":2}
final2["Furnishing"]=final2["Furnishing"].map(a)
b={"Well below average":0,"Slightly below average":1,"Slightly above average":2,"Well above average":3}
final2["Crime_Rate"]=final2["Crime_Rate"].map(b)
c={"Low":0,"Medium":1,"High":2}
final2["Dust_and_Noise"]=final2["Dust_and_Noise"].map(c)

In [ ]:
plt.scatter(final2.loc[:,"Crime_Rate"],final2.loc[:,"Neighborhood_Review"])

In [ ]:
final2.iloc[:,[10,-1]]

In [ ]:
ddf = final2.iloc[:,[10,-1]]
tbs = ddf.loc[ddf["Crime_Rate"].notnull()]
tp =  ddf.loc[ddf["Crime_Rate"].isnull()]
ts = tp.iloc[:,1]
pred = lore(tbs,pd.DataFrame(ts))
tp.iloc[:,0]=pred
tp

In [ ]:
CR_filled = pd.concat([tbs,tp]).sort_index()
CR_filled

In [ ]:
b={0:"Well below average",1:"Slightly below average",2:"Slightly above average",3:"Well above average"}
CR_filled["Crime_Rate"]=CR_filled["Crime_Rate"].map(b)
CR_filled

In [ ]:
ddf = final2.iloc[:,[-3,-2]]
tbs = ddf.loc[ddf["Dust_and_Noise"].notnull()]
tp =  ddf.loc[ddf["Dust_and_Noise"].isnull()]
ts = tp.iloc[:,1]
pred = lore(tbs,pd.DataFrame(ts))
tp.iloc[:,0]=pred
tp

In [ ]:
dan_filled=pd.concat([tbs,tp]).sort_index()
dan_filled

In [ ]:
a={0:"Unfurnished",1:"Semi_Furnished",2:"Fully Furnished"}
dan_filled["Dust_and_Noise"]=dan_filled["Dust_and_Noise"].map(a)
dan_filled

In [ ]:
f_filled = final.iloc[:,[5]].ffill()

In [ ]:
final["Number_of_Windows"]=windows_filled["Number_of_Windows"]
final["Furnishing"]=f_filled["Furnishing"]
final["Crime_Rate"]=CR_filled["Crime_Rate"]
final["Dust_and_Noise"]=dan_filled["Dust_and_Noise"]
final["Frequency_of_Powercuts"]=fop_filled["Frequency_of_Powercuts"]

In [ ]:
fin2 = final.iloc[:,[1,5,7,8,10,11]]
fin2 = ohe.fit_transform(fin2)
fin2 = pd.DataFrame(fin2.toarray())

fin3 = final.iloc[:,[2,3,4,6,9,12,13]]
fin3 = sc.fit_transform(fin3)
fin3 = pd.DataFrame(fin3)

final = pd.concat([fin2,fin3],axis=1)

In [ ]:
final.columns=[a for a in range(0,len(final.columns))]

In [ ]:
train=final[:39499]
test=final[39499:]

In [ ]:
def regressor1(x_train,x_test,y_train):
  from sklearn.linear_model import LinearRegression
  from sklearn.tree import DecisionTreeRegressor
  from sklearn.ensemble import RandomForestRegressor  
  from sklearn.svm import SVR
  from sklearn.linear_model import Ridge,Lasso,ElasticNet

  lr = LinearRegression()
  lr.fit(x_train,y_train)
  y_pred = lr.predict(x_test)
  
  dtr = DecisionTreeRegressor()
  dtr.fit(x_train,y_train)
  y_pred2 = dtr.predict(x_test)

  rfr = RandomForestRegressor()
  rfr.fit(x_train,y_train)
  y_pred3 = rfr.predict(x_test)

  svr = SVR()
  svr.fit(x_train,y_train)
  y_pred4 = svr.predict(x_test)

  r = Ridge()
  r.fit(x_train,y_train)
  y_pred5 = r.predict(x_test)
  
  lasso = Lasso()
  lasso.fit(x_train,y_train)
  y_pred6 = lasso.predict(x_test)

  en = ElasticNet()
  en.fit(x_train,y_train)
  y_pred7 = en.predict(x_test)

  return y_pred,y_pred2,y_pred3,y_pred4,y_pred5,y_pred6,y_pred7

In [ ]:
p1,p2,p3,p4,p5,p6,p7 = regressor1(train,test,y)

In [ ]:
t2 = pd.read_csv("/content/test.csv")
id = t2.iloc[:,0]
submission = pd.DataFrame({"Property_ID":id,"Habitability_score":p3})
submission.to_csv("App1.csv",index=False)
pd.read_csv("App1.csv")

#Data Cleaning : Approach 2

In [ ]:
train = pd.read_csv("/content/train.csv")

In [ ]:
test = pd.read_csv("/content/test.csv")

In [ ]:
y = train.iloc[:,-1]
train.drop(train.columns[-1],axis=1,inplace=True)

In [ ]:
final = pd.concat([train,test])
final.reset_index(inplace=True)
final=final.iloc[:,1:]
final.shape

In [ ]:
plt.scatter(final["Number_of_Doors"],final["Number_of_Windows"])

In [ ]:
final.iloc[:,[3,4]].sort_values(by="Number_of_Doors")

In [ ]:
windows_filled = final.iloc[:,[3,4]].sort_values(by="Number_of_Doors").bfill().sort_index()

In [ ]:
plt.scatter(final.iloc[:,6],final.iloc[:,-1])

In [ ]:
fop_filled = final.iloc[:,[6,-1]].sort_values(by="Neighborhood_Review").ffill().sort_index()

In [ ]:
final.iloc[:,[10,-1]].groupby("Crime_Rate").mean()

In [ ]:
CR_filled = final.iloc[:,[10,-1]].sort_values(by="Neighborhood_Review").ffill().sort_index()

In [ ]:
final2.iloc[:,[-3,-2]]

In [ ]:
final.iloc[:,[-5,-3,-2]].groupby("Dust_and_Noise").mean()

In [ ]:
dan_filled = final.iloc[:,[-3,-2]].sort_values(by="Air_Quality_Index").ffill().sort_index()

In [ ]:
miss.drop("Dust_and_Noise",axis=1,inplace=True)

In [ ]:
final.iloc[:,[1,2,5]].groupby(["Furnishing","Property_Type"]).sum().plot(kind="bar")

In [ ]:
f_filled = final.iloc[:,[5]].ffill()

In [ ]:
final["Number_of_Windows"]=windows_filled["Number_of_Windows"]
final["Furnishing"]=f_filled["Furnishing"]
final["Crime_Rate"]=CR_filled["Crime_Rate"]
final["Dust_and_Noise"]=dan_filled["Dust_and_Noise"]
final["Frequency_of_Powercuts"]=fop_filled["Frequency_of_Powercuts"]

In [ ]:
fin2 = final.iloc[:,[1,5,7,8,10,11]]
fin2 = ohe.fit_transform(fin2)
fin2 = pd.DataFrame(fin2.toarray())

fin3 = final.iloc[:,[2,3,4,6,9,12,13]]
fin3 = sc.fit_transform(fin3)
fin3 = pd.DataFrame(fin3)

final = pd.concat([fin2,fin3],axis=1)

In [ ]:
final.columns=[a for a in range(0,len(final.columns))]

In [ ]:
train=final[:39499]
test=final[39499:]

In [ ]:
p1,p2,p3,p4,p5,p6,p7 = regressor1(train,test,y)

In [ ]:
t2 = pd.read_csv("/content/test.csv")
id = t2.iloc[:,0]
submission = pd.DataFrame({"Property_ID":id,"Habitability_score":p3})
submission.to_csv("App2.csv",index=False)
pd.read_csv("App2.csv")

#Data Cleaning : Approach 3

In [ ]:
train = pd.read_csv("/content/train.csv")

In [ ]:
test = pd.read_csv("/content/test.csv")

In [6]:
y = train.iloc[:,-1]
train.drop(train.columns[-1],axis=1,inplace=True)

In [7]:
final = pd.concat([train,test])
final.reset_index(inplace=True)
final=final.iloc[:,1:]
final.shape

(49999, 14)

In [8]:
from sklearn.impute import SimpleImputer, KNNImputer

In [9]:
impute = SimpleImputer(strategy = "most_frequent")
knnimputer = KNNImputer(n_neighbors=25)

In [10]:
final2 = final.iloc[:,[3,6]]
final2 = knnimputer.fit_transform(final2)
final2 = pd.DataFrame(final2)

In [11]:
final3 = final.iloc[:,[5,10,11]]
final3 = impute.fit_transform(final3)
final3 = pd.DataFrame(final3)

In [12]:
final.iloc[:,3] = final2.iloc[:,0]
final.iloc[:,6] = final2.iloc[:,1]
final.iloc[:,5] = final3.iloc[:,0]
final.iloc[:,10] = final3.iloc[:,1]
final.iloc[:,11] = final3.iloc[:,2]

In [13]:
fin2 = final.iloc[:,[1,5,7,8,10,11]]
fin2 = ohe.fit_transform(fin2)
fin2 = pd.DataFrame(fin2.toarray())

fin3 = final.iloc[:,[2,3,4,6,9,12,13]]
fin3 = sc.fit_transform(fin3)
fin3 = pd.DataFrame(fin3)

final = pd.concat([fin2,fin3],axis=1)

In [14]:
final

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,0,1,2,3,4,5,6
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,-0.664983,0.194113,-1.128415,-0.651581,-0.382456,-0.510910,0.156310
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,-0.339236,-0.731735,-0.280217,0.701786,-1.629444,-0.412596,-0.233093
2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,-0.337158,0.027157,-0.280217,-0.651581,0.897347,-0.002955,0.093503
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,-0.252474,-0.352289,-0.280217,2.055153,-0.160952,-0.347053,-3.009160
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.442659,3.821617,3.112578,-0.651581,-0.735222,-0.084883,1.299397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49994,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,-0.220783,0.194113,0.567982,-0.651581,-0.751630,-0.445367,-0.233093
49995,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,-0.573545,-1.111181,-0.280217,3.408519,0.298465,-0.248740,-1.049583
49996,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,13.538502,0.194113,-0.280217,-0.651581,2.398655,0.455842,0.357292
49997,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.033788,-0.731735,0.567982,0.701786,-0.087117,0.160901,-0.308461


In [15]:
final.columns=[a for a in range(0,len(final.columns))]

In [16]:
train=final[:39499]
test=final[39499:]

In [ ]:
p1,p2,p3,p4,p5,p6,p7 = regressor1(train,test,y)

In [ ]:
t2 = pd.read_csv("/content/test.csv")
id = t2.iloc[:,0]
submission = pd.DataFrame({"Property_ID":id,"Habitability_score":p3})
submission.to_csv("App3.csv",index=False)
pd.read_csv("App3.csv")

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam
model = Sequential()
model.add(Dense(512, kernel_initializer='normal',activation='relu',input_shape=(31,)))
model.add(Dense(512, kernel_initializer='normal',activation='relu'))
model.add(Dense(256, kernel_initializer='normal',activation='relu'))
model.add(Dense(256, kernel_initializer='normal',activation='relu'))
model.add(Dense(128, kernel_initializer='normal',activation='relu'))
model.add(Dense(128, kernel_initializer='normal',activation='relu'))
model.add(Dense(1))

model.summary()
model.compile(loss='mse', optimizer=Adam(lr=0.0001), metrics=['mse','mae'])

In [ ]:
model.fit(train,y,epochs=30)

In [ ]:
pr = model.predict(test)

In [ ]:
pr = [a[0] for a in pr]
t2 = pd.read_csv("/content/test.csv")
id = t2.iloc[:,0]
submission = pd.DataFrame({"Property_ID":id,"Habitability_score":pr})
submission.to_csv("ann.csv",index=False)
pd.read_csv("ann.csv")

In [ ]:
ddff1 = pd.read_csv("App1.csv")
ddff2 = pd.read_csv("App2.csv")
ddff3 = pd.read_csv("App3.csv")

In [ ]:
pd.concat([ddff1,ddff2,ddff3],axis=1).iloc[:,[0,1,3,5]]

In [ ]:
dict = {"Approach 1":81.96442,"Approach 2":81.87324,"Approach 3":82.97659}

In [ ]:
pd.DataFrame(dict,index=["R2_score"]).T

In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1000, num = 5)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [18]:
from sklearn.ensemble import RandomForestRegressor

In [20]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(train,y)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 14.2min finished


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000]},
                   random_state=42, verbose=2)

In [22]:
rf_random.best_params_

{'n_estimators': 400,
 'min_samples_split': 2,
 'min_samples_leaf': 2,
 'max_features': 'auto',
 'max_depth': 60,
 'bootstrap': True}

In [24]:
rfr = RandomForestRegressor(n_estimators=400,
 min_samples_split=2,
 min_samples_leaf= 2,
 max_features= 'auto',
 max_depth= 60,
 bootstrap= True)

In [25]:
rfr.fit(train,y)

[CV] n_estimators=1000, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=30, bootstrap=False 
[CV]  n_estimators=1000, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=30, bootstrap=False, total= 5.6min
[CV] n_estimators=800, min_samples_split=2, min_samples_leaf=2, max_features=auto, max_depth=70, bootstrap=False 
[CV]  n_estimators=800, min_samples_split=2, min_samples_leaf=2, max_features=auto, max_depth=70, bootstrap=False, total= 5.1min
[CV] n_estimators=200, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=40, bootstrap=True 
[CV]  n_estimators=200, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=40, bootstrap=True, total=  11.8s
[CV] n_estimators=600, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=50, bootstrap=False 
[CV]  n_estimators=600, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=50, bootstrap=False, total= 2.9min
[CV] n_estimators=200, min

RandomForestRegressor(max_depth=60, min_samples_leaf=2, n_estimators=400)

In [26]:
predicted = rfr.predict(test)

In [30]:
t2 = pd.read_csv("/kaggle/input/usa-property-dataset/dataset/test.csv")
id = t2.iloc[:,0]
submission = pd.DataFrame({"Property_ID":id,"Habitability_score":predicted+0.55})
submission.to_csv("rfr.csv",index=False)
pd.read_csv("rfr.csv")


,Property_ID,Habitability_score
0,0x6e93,31.124495
1,0x8787,81.105395
2,0x6c17,67.405667
3,0x9dbd,74.372273
4,0xbfde,77.035379
...,...,...
10495,0x423d,64.729123
10496,0x78c5,81.497930
10497,0xbf3,75.239967
10498,0x305b,80.269967
